# Eli Lilly - Accumulations
Before start, look at `1. List`

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [3]:
# Read pdf into DataFrame
df_list = tabula.read_pdf("EFPIA_2017_Switzerland_edited.pdf", pages='all', lattice=True, multiple_tables=True)
df = pd.concat(df_list)

## Format Table

In [20]:
df_export = df.copy()

#Shift
column = df_export.columns[0]
df_export[df_export[column].isna()] = df_export[df_export[column].isna()].shift(-1, axis='columns')

#Drop Columns
df_export = df_export.drop(df.columns[9:],axis=1)
df_export = df_export.drop(df.columns[7],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

# Select
amount = df_export[df_export.type.str.contains('Aggregate amount', na=False)]
recipients = df_export[df_export.type.str.contains('Number of Recipients', na=False)]
rnd = df_export[df_export.type.str.contains('Transfers of Value re', na=False)]

df_export = pd.concat([amount, recipients, rnd])

#Fill N/A
df_export = df_export.replace('N/A', '')

#Numberize
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"
df_export.loc[df_export.type == 'rnd', "total"] = df_export.sponsorship
df_export.loc[df_export.type == 'rnd', "sponsorship"] = np.NaN

export_acumulations(df_export, 'eli')

Total nicht Summe der Werte
saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx', open=True)